In [270]:
from SIV_library.lib import block_match, moving_window_array, correlation_to_displacement, get_field_shape, get_x_y, SIVDataset
from torchPIV.PIVbackend import OfflinePIV

import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [332]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

dataset = SIVDataset("../Test Data/plume simulation_PROCESSED", None)  # (300, 2500, 400, 1600)

cuda


In [340]:
def vector_field(lib_res, corr_res, sad_res) -> None:
    fig, (lib, corr, sad) = plt.subplots(1, 3)
 
    frame = dataset[0][0].cpu().numpy()
    lib.imshow(frame, cmap='gray')
    corr.imshow(frame, cmap='gray')
    sad.imshow(frame, cmap='gray')
    
    lib_res[1] = np.flip(lib_res[1], axis=0)
    corr_res[3] *= -1
    sad_res[3] *= -1
    
    lib.quiver(*lib_res, color='red', scale=3, scale_units='xy')
    corr.quiver(*corr_res, color='red', scale=1, scale_units='xy')
    sad.quiver(*sad_res, color='red', scale=0.3, scale_units='xy')
    
    lib.set_title('TorchPIV')
    corr.set_title('Correlation')
    sad.set_title('SAD')
    
    lib.set_axis_off()
    corr.set_axis_off()
    sad.set_axis_off()
    
    fig.tight_layout()
    plt.show()

In [350]:
def all_frames(window_size, overlap):
    piv_gen = OfflinePIV(
                folder="../Test Data/plume simulation_PROCESSED",
                device=torch.cuda.get_device_name(),
                file_fmt="jpg",
                wind_size=window_size,
                overlap=overlap,
                dt=12,
                scale = 1.,
                multipass=2,
                multipass_mode="DWS",
                multipass_scale=2.,
                folder_mode="sequential"
            )
    piv_gen._dataset = dataset
    
    for idx, data in enumerate(dataset):
        if idx == 0:
            image_a, image_b = data
            image_a, image_b = image_a.to(device), image_b.to(device)
            
            n_rows, n_cols = get_field_shape(image_a.shape, window_size, overlap)
            window = moving_window_array(image_a, window_size, overlap)
            
            area = moving_window_array(image_b, window_size, overlap, area=(30, 30, 30, 30))
            
            lib = next(piv_gen())
            
            corr = block_match(window, area, 0)
            u_c, v_c = correlation_to_displacement(corr, n_rows, n_cols, 0)
            
            sad = block_match(window, area, 1)
            u_s, v_s = correlation_to_displacement(sad, n_rows, n_cols, 1)
    
            x, y= get_x_y(image_a.shape, window_size, overlap)
            x, y = x.reshape(n_rows, n_cols), y.reshape(n_rows, n_cols)
            
            lib_res, corr_res, sad_res = np.array(lib), np.array((x, y, u_c, v_c)), np.array((x, y, u_s, v_s))
    return lib_res, corr_res, sad_res

In [351]:
lib_res, corr_res, sad_res = all_frames(64, 32)

Load time 0.044 sec Iteration finished in 0.115 sec 

SAD: 100%|██████████| 61/61 [00:18<00:00,  3.36it/s]


In [352]:
vector_field(lib_res, corr_res, sad_res)